In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation, metrics


import xgboost
from xgboost import DMatrix

/Users/Roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("../datasets/train_base.csv")
test = pd.read_csv("../datasets/test_base.csv")

In [3]:
train.head(2)

,client_id,gender,age,credit_sum,credit_month,tariff_id,score_shk,living_region,monthly_income,credit_count,...,job_position_WRP,marital_status_CIV,marital_status_DIV,marital_status_MAR,marital_status_UNM,marital_status_WID,credit_pay,money_for_life,dif_city,target
0,1,1,48,59998.0,10,1.6,0.770249,37297.641479,30000.0,1.0,...,0,0,0,1,0,0,5999.800000,24000.200000,-7297.641479,0
1,2,0,28,10889.0,6,1.1,0.248514,57219.334894,43000.0,2.0,...,0,0,0,1,0,0,1814.833333,41185.166667,-14219.334894,0


In [4]:
y = train.target
train.drop("target", axis=1, inplace=True)
train.drop("client_id", axis=1, inplace=True)

ans = test[["client_id"]]
test.drop("client_id", axis=1, inplace=True)

In [5]:
Dtrain = DMatrix(data=train.to_sparse(), label=y)

In [6]:
seed = 0
np.random.seed(seed=seed)

In [9]:
param = {
    "max_depth" : 5,
    "eta" : 0.15,
    "objective": "binary:logistic",
    "eval_metric": "auc",
}

numround = 400
history = xgboost.cv(params=param, dtrain=Dtrain, num_boost_round=numround, nfold=3, stratified=True,\
                     early_stopping_rounds=25, verbose_eval=25, seed=seed)

[0]	train-auc:0.71423+0.00190277	test-auc:0.710527+0.0014376
[25]	train-auc:0.768292+0.00031682	test-auc:0.757606+0.00128008
[50]	train-auc:0.780066+0.000285646	test-auc:0.762935+0.0006824
[75]	train-auc:0.786699+0.00133588	test-auc:0.764337+0.000812095
[100]	train-auc:0.791638+0.000864781	test-auc:0.764913+0.00099811
[125]	train-auc:0.796025+0.000987236	test-auc:0.765381+0.00107375
[150]	train-auc:0.800411+0.00104445	test-auc:0.7655+0.000935577
[175]	train-auc:0.804273+0.00146155	test-auc:0.765378+0.000956299


In [ ]:
history.tail(2)

In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
def objective(space):
    print(space)
    
    
    
    
    res = metrics.roc_auc_score(y_test, y_ans)
    print(res)
    return{'loss': -res, 'status': STATUS_OK}


space ={
        'n_estimators' : hp.quniform('n_estimators', 100, 600, 1),
        'eta' : hp.uniform('eta', 0.01, 0.4),
        'max_depth' : hp.quniform('max_depth', 3, 10, 1),
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print(best)

In [ ]:
print test.shape

In [10]:
param = {
    "max_depth" : 5,
    "eta" : 0.12,
    "objective": "binary:logistic",
    "eval_metric": "auc",
}

numround = 160
bst = xgboost.train(params=param, dtrain=Dtrain, num_boost_round=numround)

In [11]:
y_ans = bst.predict(DMatrix(data=test.to_sparse()))
ans['val'] = y_ans
ans.rename(columns={"client_id":"_ID_", "val":"_VAL_"}, inplace=True)
ans.to_csv("../solutions/xgboost_base_test.csv", index=False, sep=",", header=False)

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
